# **2_task_veri_hazirlama**

# **2.1 Gerekli Kütüphanelerin Yüklenmesi**

### Neden Önemli?
Veri ön işleme aşamasında kullanılacak tüm fonksiyonları, grafik araçlarını ve veri dönüşüm araçlarını bu kütüphaneler sağlar.

### Bu bölümde ne yapacağız?
- **`NumPy, pandas, seaborn, sklearn`** gibi paketleri import edeceğiz.
- **Veri ön işleme** sırasında kullanacağımız fonksiyonları tanımlayacağız.

In [ ]:
import numpy as np                 # Sayısal işlemler
import pandas as pd                # Veri çerçeveleri ile çalışma
import seaborn as sns              # Veri görselleştirme
import matplotlib.pyplot as plt
from matplotlib.cbook import boxplot_stats  # Kutu grafiği istatistikleri
import random                      # Rastgele işlemler
from random import sample          # Belirli bir listeden rastgele seçim
from sklearn.model_selection import train_test_split  # Eğitim/Test veri bölme
from sklearn.preprocessing import MinMaxScaler, StandardScaler  # Normalizasyon & standardizasyon


print("✅ Gerekli kütüphaneler başarıyla yüklendi.")



---



---



---



# **2.2 Veri Okuma**

### Neden Önemli?
Veriyi doğru biçimde okumak ve bir **`DataFrame`** hâline getirmek, sonraki tüm veri ön işleme ve analiz adımları için kritik bir adımdır. Eğer veri yanlış bir konumdan okunursa veya sütun başlıkları hatalı tanımlanırsa, ileriki aşamalarda eksik ya da hatalı sonuçlar alabiliriz.

### Bu bölümde ne yapacağız?
- Google Drive’ı bağlayarak **(`mount`)** veri setinin bulunduğu klasöre erişeceğiz.

- **`auto-mpg.data`** dosyasını, pandas’ın **`read_csv`** fonksiyonunu kullanarak okuyacağız.
- Dosyada başlık **(`header`)** bulunmadığı için names parametresiyle sütun isimlerini manuel tanımlayacağız.
- Veri setinin sütun veri tiplerini **(`dtypes`)**inceleyeceğiz.
- Ayrıca **`head()`** fonksiyonuyla veri içeriğine hızlıca göz atacağız.

In [ ]:
# 1) Google Drive'ı mount ediyoruz
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# 2) Sütun isimlerini belirliyoruz
nitelikAdlari = [
    "mpg", "cylinders", "displacement", "horsepower", "weight",
    "acceleration", "model_year", "origin", "car_name"
]

# 3) 'auto-mpg.data' dosyasını, Colab Notebooks klasöründen tam yol ile okuyoruz
veriSeti = pd.read_csv(
    "/content/drive/MyDrive/Colab Notebooks/auto-mpg.data",  # Dosyanın tam yolu
    sep=r"\s+",        # Boşluk veya tab ayracı
    header=None,       # Dosyada sütun başlığı yok
    names=nitelikAdlari,  # Yukarıda tanımlanan sütun isimleri
    decimal="."        # Ondalık ayırıcı
)

# 4) Veri setinin boyutu ve sütun veri tipleri
print("Sütun Veri Tipleri:\n", veriSeti.dtypes)
display(veriSeti.head(10))




---


---



---



# **2.3 Eksik Veri Tamamlama (Missing Data Imputation)**

### Neden Önemli?
Eksik veriler, modelin doğruluğunu olumsuz etkileyebilir veya hata oluşturabilir. Bu nedenle eksik verileri uygun yöntemlerle doldurmak **(`imputation`)**, veri kalitesini artırır ve analiz veya makine öğrenmesi modellerinden daha güvenilir sonuçlar elde etmemize yardımcı olur.

### Bu bölümde ne yapacağız?
- **`horsepower`** sütunundaki **'?'** karakterlerini **`NaN`’e** dönüştüreceğiz.
- Bu sütunun veri tipini sayısal **`(float64)`** hâline getireceğiz.
- Eksik veri sayısını inceleyip, ortalama değerle dolduracağız.
- **`fillna()`** metodu kullanımını örnek olarak göreceğiz.

In [ ]:
soru_isareti_sayisi = (veriSeti["horsepower"] == "?").sum()       # Ön kontrol: horsepower sütununda "?" karakteri var mı?
print("horsepower sütununda '?' sayısı:", soru_isareti_sayisi)    # Bu satır, "?" işareti olan satırların sayısını ekrana basar.

# 1) '?' karakterini NaN'e dönüştürme
veriSeti.loc[veriSeti.horsepower == "?", "horsepower"] = np.nan   # loc[] fonksiyonu, koşul ile belirtilen satırlara erişmemizi sağlar.
                                                                  # horsepower sütununda "?" görülen hücreleri NaN (eksik değer) işaretleme.
# 2) 'horsepower' sütununun tipini float64'e dönüştürme
veriSeti.horsepower = veriSeti.horsepower.astype("float64")       # astype("float64"), sütun tipini sayısal hale getirir.

# 3) Eksik veri sayısını (ilk kontrol) ekrana bastırma
print("Eksik Veri Sayısı (Önce):\n", veriSeti.isnull().sum())     # isnull().sum(), sütun bazında NaN sayısını verir.

# 4) fillna() metodu ile ortalamayla doldurma
veriSeti["horsepower"].fillna(
    veriSeti["horsepower"].mean().round(0),                       # mean().round(0), sütunun ortalamasını en yakın tam sayıya yuvarlar.
    inplace=True                                                  # inplace=True, değişikliği doğrudan veriSeti üzerinde uygular.
)
print("Eksik Veri Sayısı (Sonra):\n", veriSeti.isnull().sum())    # Eksik veri sayısını tekrar kontrol ediyoruz



---



---



---



# **2.4 Veri Ayırıklaştırma (Binning / Discretization)**

### Neden Önemli?
- Sürekli (sayısal) veriler, kategorik (gruplandırılmış) değerlere dönüştürülür.
- Bu dönüşümle, veriye yeni bir sütun eklenir veya mevcut sütun sınıflara ayrılır.
- Amaç: Sayısal değerleri daha anlamlı ve genellenmiş kategorilere indirgemektir.

Örneğin: `mpg` değişkeni;
- **23.5’in altında** ise → **Düşük**
- **23.5 – 30 arası** ise → **Orta**
- **30 ve üstü** ise → **Yüksek** olarak sınıflandırılabilir.

Bu işlem sonucunda:
- Veri setine **yeni bir kategorik sütun (örneğin: `durum`)** eklenir.
- Satır sayısı değişmez; sadece her satır bir kategoriye atanır.

---

### Bu bölümde ne yapacağız?
- `mpg` sütununu **Düşük**, **Orta** ve **Yüksek** kategorilerine ayıracağız.
- Dört farklı yöntemle ayrıklaştırma işlemi gerçekleştireceğiz:

I. Yöntem:

 **1.** **`lambda` + `map()`** fonksiyonu ile manuel koşullu sınıflandırma yapacağız.  

II. Yöntem:

**2.1.** **`pd.cut()`** fonksiyonu ile **sabit aralık (fixed interval)** yöntemi uygulayacağız.  
**2.2.** **`pd.qcut()`** fonksiyonu ile **eşit frekans (quantile)** yöntemi kullanacağız.  
**2.3.** **`pd.cut()`** fonksiyonu ile **eşit genişlik (equal interval)** yöntemini uygulayacağız.


Her yöntemde amaç, `mpg` değerlerini farklı sınıflandırma stratejileriyle gruplandırmak ve analizlerde kullanılabilecek kategorik yapılar oluşturmaktır.


In [ ]:
# ------------------------------------------------------------
# I. Yöntem: lambda + map
# ------------------------------------------------------------
# lambda ifadesi, 'mpg' değerine göre koşullu atama yapar:
# - x < 23.5 ise "Düşük"
# - 23.5 <= x < 30 ise "Orta"
# - x >= 30 ise "Yüksek" olarak etiketler.
veriSeti["durum"] = veriSeti.mpg.map(
    lambda x: "Düşük" if x < 23.5 else ("Orta" if (x >= 23.5 and x < 30) else "Yüksek")
).astype("category")

print("Durum (lambda) dağılım:\n", veriSeti.durum.value_counts())
print(f"\n{'-'*82}\n")













# ------------------------------------------------------------
# 🟩 1.GÖREV (lambda + map uygulaması)
# ------------------------------------------------------------
# mpg değişkenini aşağıdaki aralıklara göre etiketleyin:
# - x < 17           → "Çok Düşük"
# - 17 <= x < 24     → "Düşük"
# - 24 <= x < 32     → "Orta"
# - x >= 32          → "Yüksek"














# ------------------------------------------------------------
# # 🟨 1.CEVAP ANAHTARI - lambda + map
# ------------------------------------------------------------
veriSeti["durum_gorev"] = veriSeti.mpg.map(
    lambda x: "Çok Düşük" if x < 17 else ("Düşük" if 17 <= x < 24 else "Orta" if 24 <= x < 32 else "Yüksek")
).astype("category")

print("1.Görev (lambda + map) dağılım:\n", veriSeti.durum_gorev.value_counts())
print(f"\n{'-'*82}\n")

















#----------------------------------------------------------------------------------
# II. Yöntem:
#----------------------------------------------------------------------------------
# II.1: Yöntem: Fixed (sabit) aralıklarla bölme
bolmeKategorileri = ["Düşük", "Orta", "Yüksek"]   # önceden belirlenmiş sınırlar kullanılarak 'mpg' değerleri üç kategoriye ayrılır.
bolmeler = [8, 23.4, 29.9, 46.6]                  # Belirlenen sınırlar

veriSeti["durum"] = pd.cut(
    veriSeti["mpg"],
    bins=bolmeler,
    labels=bolmeKategorileri
)
print("Durum (fixed cut) dağılım:\n", veriSeti.durum.value_counts())
print(f"\n{'-'*82}\n")



















# ------------------------------------------------------------
# 🟩 2.GÖREV (Fixed – sabit aralıklarla sınıflandırma)
# ------------------------------------------------------------

# sabit_labels:  ["Çok Düşük", "Düşük", "Orta", "Yüksek"]
# sabit_bins:  [8, 16.9, 23.9, 31.9, 47]
















# ------------------------------------------------------------
# 🟨 2.CEVAP ANAHTARI – pd.cut (fixed interval)
# ------------------------------------------------------------
sabit_labels = ["Çok Düşük", "Düşük", "Orta", "Yüksek"]
sabit_bins   = [8, 16.9, 23.9, 31.9, 47]

veriSeti["durum_gorev_cut"] = pd.cut(
    veriSeti["mpg"],
    bins=sabit_bins,
    labels=sabit_labels
)

print("2.Görev (fixed cut) dağılım:\n", veriSeti.durum_gorev_cut.value_counts())
print(f"\n{'-'*82}\n")





















# ------------------------------------------------------------
# II.2: Yöntem: Equal Frequency (qcut)
# 'mpg' değerleri yaklaşık eşit sayıda gözlem içerecek şekilde üç gruba ayrılır.

qcut_frekans_etiketleri = ["Düşük", "Orta", "Yüksek"]
durum_ef = pd.qcut(
    veriSeti["mpg"],
    q=3,
    labels=qcut_frekans_etiketleri
)

print("Durum (Eşit frekans) dağılım:\n", durum_ef.value_counts())
print(f"\n{'-'*82}\n")
# ------------------------------------------------------------






















# ------------------------------------------------------------
# 🟩 3. GÖREV (Eşit Frekans – pd.qcut uygulaması)
# ------------------------------------------------------------
# mpg değişkenini eşit frekanslı 4 kategoriye ayırınız.
# Kategori etiketleri: ["Çok Düşük", "Düşük", "Orta", "Yüksek"]
























# ------------------------------------------------------------
# 🟨 3.CEVAP ANAHTARI – pd.qcut (equal frequency)
# ------------------------------------------------------------
qcut_frekans_etiketleri = ["Çok Düşük", "Düşük", "Orta", "Yüksek"]

gorev_durum_ef = pd.qcut(
    veriSeti["mpg"],
    q=4,
    labels=qcut_frekans_etiketleri
)

print("3.Görev (eşit frekans) dağılım:\n", gorev_durum_ef.value_counts())
print(f"\n{'-'*82}\n")
# ------------------------------------------------------------






















# ------------------------------------------------------------
# II.3: Yöntem: Equal Interval (eşit genişlik)
# 'mpg' değerlerinin aralığı eşit genişlikli üç dilime bölünür.

cut_frekans_etiketleri = ["Düşük", "Orta", "Yüksek"]

durum_ea = pd.cut(
    veriSeti["mpg"],
    bins=3,
    labels=cut_frekans_etiketleri
)

print("Durum (Eşit Genişlik) dağılım:\n", durum_ea.value_counts())
print(f"\n{'-'*82}\n")











---



---



---



# **2.5 Veri Seti Özeti (describe)**

### Neden Önemli?
Veri setinin genel istatistiksel özelliklerini (ortalama, std, min, max vb.) görmek, veri hakkında hızlıca fikir sahibi olmamızı sağlar. Hem sayısal hem de kategorik sütunların özetini almak, veri analizinin sonraki adımlarını planlamada bize rehberlik eder.

### Bu bölümde ne yapacağız?
- pd.set_option("display.max_columns", 20) ayarıyla, DataFrame ekrana basılırken en fazla kaç sütun gösterileceğini belirleyeceğiz.

- veriSeti.describe(include="all") fonksiyonunu kullanarak, tüm sütunların (sayısal ve kategorik) temel istatistiksel özetini görüntüleyeceğiz.

In [ ]:
# 1) pd.set_option ile ekrana basılacak sütun sayısını 20 ile sınırlıyoruz
pd.set_option("display.max_columns", 20)

# 2) describe(include="all") ile sayısal ve kategorik tüm sütunların özet istatistiklerini alıyoruz
print("Veri Seti Özeti (describe, include='all'):\n", veriSeti.describe(include="all"))


# **2.6 Veri Gruplandırma (Aggregate)**

### Neden Önemli?
Makine öğrenmesi ve veri bilimi süreçlerinde, bazen tüm veri setini tek bir bütün olarak değerlendirmek yerine, verileri belirli özelliklere göre alt gruplara ayırıp ayrı ayrı incelemek faydalıdır. Bu işlem sayesinde her bir alt grup hakkında istatistiksel özet bilgiler elde edebiliriz. Bu, veriyi anlamak, farklı alt grupları karşılaştırmak ve karar vermek açısından önemlidir.

**Örneğin:** Bir otomobil verisinde araçların yakıt tüketimini (mpg) "durum" kategorisine (Düşük, Orta, Yüksek) göre ayrı ayrı inceleyebilirsiniz. Böylece her grubun yakıt tüketim özelliklerini ayrı ayrı değerlendirebilirsiniz.


### **Veri Ayrıklaştırma ile Gruplandırma Arasındaki Fark**

**Veri Ayrıklaştırma (Discretization) = sayısalı kategorik hale getirme**
- **Amaç:** Sürekli değişkenleri kategorik hale getirerek sınıflandırmak.
- **Ön koşul:** Kategorik bir sütun yoksa, analiz için kendimiz üretmeliyiz.

**Veri Gruplandırma (Aggregation) = zaten var olan kategorik değişkene göre istatistik çıkarmak**
- **Amaç:** Var olan kategorik sütuna göre gruplar oluşturup bu gruplar için istatistikler hesaplamak.
- **Ön koşul:** Veri setinde zaten kategorik bir değişken (örneğin durum) varsa doğrudan kullanılabilir.r.



### Bu bölümde ne yapacağız?
- **groupby("durum")** ifadesini kullanarak, durum sütunu bazında veri setini gruplara ayıracağız.

- Farklı toplulaştırma fonksiyonları **(mean, sum, count, min, max, std, describe)** ile her bir kategorinin temel istatistiklerini görüntüleyeceğiz.

In [ ]:
# 1. Adım: "durum" kategorisine göre "mpg" değişkeninin temel istatistiklerini çıkar
df_ozet = veriSeti.groupby("durum")["mpg"].describe().round(2)


# 2. Adım: 'durum' değişkeni indeks olarak değil, sütun olarak görünsün diye resetlenir
df_ozet.reset_index(inplace=True)
display(df_ozet)













# ------------------------------------------------------------
# 🟩 4.GÖREV: horsepower → sabit aralıklarla guc_durumu oluştur ve horsepower'ı grupla
# ------------------------------------------------------------
# 1. horsepower değişkenini sabit sınırlarla 3 kategoriye ayırınız:

#    - guc_etiketleri = ["Düşük Güç", "Orta Güç", "Yüksek Güç"]
#    - guc_sinirlar: [45, 85, 125, 250]

# 2. Yeni sütun adı: guc_durumu
#
# 3. guc_durumu sütununa göre horsepower değerlerini gruplayarak
#    .describe() fonksiyonu ile özet istatistikleri hesaplayınız.
#
# 4. Sonucu iki ondalık basamakla yazdırınız (display kullanılabilir).
# ------------------------------------------------------------
























# ------------------------------------------------------------
#🟨 4.İPUCU: # II.1: Yöntem: Fixed (sabit) aralıklarla bölme
# ------------------------------------------------------------
# bolmeKategorileri = ["Düşük", "Orta", "Yüksek"]   # önceden belirlenmiş sınırlar kullanılarak 'mpg' değerleri üç kategoriye ayrılır.
# bolmeler = [8, 23.4, 29.9, 46.6]                  # Belirlenen sınırlar

#veriSeti["durum"] = pd.cut(
#    veriSeti["mpg"],
#    bins=bolmeler,
#   labels=bolmeKategorileri
#)
# print("Durum (fixed cut) dağılım:\n", veriSeti.durum.value_counts())
















# ------------------------------------------------------------
# 🟨 4.CEVAP ANAHTARI: horsepower → sabit aralıklarla guc_durumu oluştur ve horsepower'ı grupla
# ------------------------------------------------------------

# Etiket ve sınırları tanımla
guc_etiketleri = ["Düşük Güç", "Orta Güç", "Yüksek Güç"]
guc_sinirlar   = [45, 85, 125, 250]

# horsepower → kategorik hale getir
veriSeti["guc_durumu"] = pd.cut(
    veriSeti["horsepower"],
    bins=guc_sinirlar,
    labels=guc_etiketleri
)

# horsepower'ı guc_durumu'na göre grupla
df_guc_ozet = veriSeti.groupby("guc_durumu")["horsepower"].describe().round(2)

# kategorik sütunu görünür kıl
df_guc_ozet.reset_index(inplace=True)

# çıktı
display(df_guc_ozet)











## 📊 `durum` Değişkenine Göre `mpg` (Yakıt Verimliliği) İstatistikleri

Aşağıdaki tablo, `mpg` (mil per galon) sayısal değişkeninin, `durum` adlı kategorik değişkene göre gruplandırılarak (aggregation) temel istatistiklerle özetlenmesini göstermektedir.

| Kategori (`durum`) | Gözlem | Ortalama | Std. Sapma | Min  | Q1 (%25) | Medyan (%50) | Q3 (%75) | Max  |
|--------------------|--------|----------|------------|------|----------|---------------|----------|------|
| **Düşük**          | 208    | 17.26    | 3.35       | 9.0  | 14.38    | 17.55         | 20.00    | 23.2 |
| **Orta**           | 98     | 26.34    | 1.78       | 23.5 | 25.00    | 26.00         | 28.00    | 29.9 |
| **Yüksek**         | 92     | 34.63    | 3.83       | 30.0 | 31.88    | 33.90         | 36.55    | 46.6 |

---


### Yorum:

1. **Yüksek** `durum` kategorisi:
   - En yüksek **ortalama** `mpg` değeriyle (`34.63`) en verimli gruptur.
   - Aynı zamanda **en geniş aralığa** (min: 30.0 – max: 46.6) sahip → Bu grupta farklı düzeylerde yüksek verimliliğe sahip araçlar bulunur.

2. **Orta** kategori:
   - **En düşük standart sapmaya** (`1.78`) sahiptir → Değerler birbirine çok yakındır, grup oldukça **homojen**dir.
   - Ortalama (`26.34`) ve medyan (`26.00`) birbirine çok yakındır → Dağılım simetrik olabilir.

3. **Düşük** kategori:
   - Ortalama değer (`17.26`) en düşük olan gruptur.
   - Standart sapmanın (`3.35`) yüksek olması → Gruptaki araçlar arasında önemli **çeşitlilik** vardır.
   - Alt çeyrek (`Q1 = 14.38`) ve üst çeyrek (`Q3 = 20.00`) arasındaki mesafe daha büyüktür → Daha yaygın ve düzensiz bir dağılım gösterir.

---
---






---



---



---



# **2.7 Tekrarlayan Satırların Bulunması (Duplicated Rows)**

### Neden Önemli?
Tekrarlayan **(duplicate)** satırlar, veri analizi sonuçlarını yanıltabilir. Aynı gözlemin veri setinde birden fazla kez yer alması, ortalama ve diğer istatistiksel hesaplamaları etkileyerek hatalı sonuçlara yol açabilir. Bu nedenle tekrarlayan satırları tespit edip, gereksiz olanları temizlemek veri kalitesini artırır.

### Bu bölümde ne yapacağız?
- `duplicated()` fonksiyonunu kullanarak tekrarlayan satırları bulacağız.
- `drop_duplicates()` ile tekrarlayanları temizleyeceğiz.
- Tekrarlayan satırları bulmak için farklı keep parametreleri **(örneğin "first", "last")** deneyeceğiz.

In [ ]:
# 1) duplicated(keep="first") ve duplicated(keep="last") ile tekrarlayan satırları tespit ediyoruz
    # keep="first" => ilk tekrar eden satırı False, diğer tekrarlarını True olarak işaretler
    # keep="last"  => son tekrar eden satırı False, diğer tekrarlarını True olarak işaretler
tekrarlar_f = veriSeti.duplicated(keep="first")
tekrarlar_l = veriSeti.duplicated(keep="last")

# 2) Herhangi birinin True olduğu satırlar, tekrarlayan satırlardır
indislerim = tekrarlar_f | tekrarlar_l

# 3) Tekrarlayan gözlemleri ekrana basalım
print("Tekrarlayan Gözlemler:\n", veriSeti[indislerim])

# 4) drop_duplicates() ile veri setinden tekrar eden satırları kaldırıyoruz
veriSeti2 = veriSeti.drop_duplicates()

# 5) Kaldırma işleminden sonra veri setinin boyutunu (shape) inceleyerek değişikliği gözlemleyebiliriz
print("\nTekrarlayanlardan arındırılmış veriSeti2 shape:", veriSeti2.shape)


# **2.8 Aykırı Değer Analizi (IQR ve Boxplot)**

### Aykırı Değer Nedir?

Aykırı değer, veri setindeki diğer değerlerden **belirgin şekilde uzak** olan gözlemlerdir.  
Bu değerler çoğu zaman hatalı girişler, ölçüm hataları veya sıra dışı durumları temsil edebilir.

> Örnek: Eğer `horsepower` sütunundaki değerlerin çoğu 50–150 arasında ise, 230 gibi bir değer aykırı olabilir.

---

### Neden Önemlidir?

- Aykırı değerler analizlerin sonucunu **yanıltabilir**.
- Ortalama, standart sapma gibi istatistikleri bozar.
- Bazı algoritmalar (örneğin KNN, k-means) aykırı değerlerden çok etkilenir.

---

### Aykırı Değer Nasıl Bulunur?

**1. Görsel yöntem:** Kutu grafiği (boxplot)  
**2. İstatistiksel yöntem:** IQR (Interquartile Range) yöntemi
Bu derste, `horsepower` değişkenini örnek alarak hem istatistiksel hem görsel yöntemle aykırı değerleri belirleyeceğiz.








In [ ]:
# 1. Kartil (Q1) ve 3. Kartil (Q3) değerlerini hesaplayalım
q1 = veriSeti["horsepower"].quantile(0.25)  # Alt %25'lik değeri temsil eder
q3 = veriSeti["horsepower"].quantile(0.75)  # Üst %25'lik değeri temsil eder

# IQR (çeyrekler arası açıklık): Q3 - Q1
IQR = q3 - q1

# Alt ve üst sınırları belirliyoruz
alt = q1 - 1.5 * IQR
ust = q3 + 1.5 * IQR

print(f"Aykırı değer alt sınırı: {alt}, üst sınırı: {ust}")




In [ ]:
# Aykırı olan satırların indekslerini buluyoruz
ust_aykiriDegerInd = np.where(veriSeti["horsepower"] >= ust)[0]   # Üst sınırın üzerindeki indeksler
alt_aykiriDegerInd = np.where(veriSeti["horsepower"] <= alt)[0]   # Alt sınırın altındaki indeksler

# Üst sınırın üzerindeki aykırı değerleri indeks : değer formatında yazdırıyoruz
print("Üst sınırın üzerindeki aykırı değerler:")
for i in ust_aykiriDegerInd:
    print(f"İndeks: {i}, Değer: {veriSeti['horsepower'][i]}")

# Alt sınırın altındaki aykırı değerleri indeks : değer formatında yazdırıyoruz
print("\nAlt sınırın altındaki aykırı değerler:")
for i in alt_aykiriDegerInd:
    print(f"İndeks: {i}, Değer: {veriSeti['horsepower'][i]}")


### 📊 Kutu Grafiği

- **Kutu grafiği (boxplot)**, verinin dağılımını ve olası aykırı değerleri görsel olarak anlamamızı sağlar.
- Orta çizgi: **Ortanca (medyan)**
- Kutunun alt ve üst kenarları: **1. ve 3. çeyrek (Q1, Q3)**
- Alt ve üst "bıyık" çizgileri:
  - Alt sınır = Q1 - 1.5 * IQR
  - Üst sınır = Q3 + 1.5 * IQR
- Bu sınırların **dışındaki daireler**, **aykırı değerleri** gösterir.

Bu grafik sayesinde:
- Hangi değerlerin aykırı olduğunu görselleştiririz.
- Alt sınırın altında aykırı değer olup olmadığını,
- Üst sınırın üzerinde ne kadar aykırı değer olduğunu kolayca gözlemleyebiliriz.


In [ ]:
# 1️⃣ Basit Kutu Grafiği

import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 5))
sns.boxplot(y="horsepower", data=veriSeti, palette="magma")
plt.title("Horsepower Niteliğine Ait Kutu Grafiği")
plt.show()


In [ ]:
#@title 2️⃣ Açıklamalı Kutu Grafiği

plt.figure(figsize=(10, 7))

# Aykırı değerleri gizleyerek kutu grafiği çiz
sns.boxplot(y="horsepower", data=veriSeti, palette="magma", showfliers=False)
plt.title("Gerçek Verilerle Horsepower Değişkenine Ait Açıklamalı Kutu Grafiği")

# İstatistik değerleri
ortalama = veriSeti["horsepower"].mean()
medyan = veriSeti["horsepower"].median()
q1 = veriSeti["horsepower"].quantile(0.25)
q3 = veriSeti["horsepower"].quantile(0.75)
iqr = q3 - q1
alt = q1 - 1.5 * iqr
ust = q3 + 1.5 * iqr

# Ortalama (turuncu)
plt.plot([0], [ortalama], "o-", color="orange")
plt.text(0.1, ortalama, f"Ortalama:\n{ortalama:.2f}", color="orange", fontsize=10)

# Medyan (siyah nokta)
plt.plot(0, medyan, "ko")
plt.text(-0.35, medyan, f"Medyan\n{medyan:.2f}", fontsize=10)

# Q1
plt.plot(0, q1, "ks")
plt.text(-0.35, q1, f"Q1 (Alt Çeyrek)\n{q1:.2f}", fontsize=9)

# Q3
plt.plot(0, q3, "ks")
plt.text(-0.35, q3, f"Q3 (Üst Çeyrek)\n{q3:.2f}", fontsize=9)

# Alt sınır (mavi kare)
plt.plot(0, alt, "bs")
plt.text(0.15, alt + 2, f"Alt Sınır\n{alt:.2f}", color="blue", fontsize=9)

# Üst sınır (mavi kare)
plt.plot(0, ust, "bs")
plt.text(0.15, ust + 2, f"Üst Sınır\n{ust:.2f}", color="blue", fontsize=9)

# Aykırı değerler (siyah daire)
for i, val in enumerate(veriSeti["horsepower"]):
    if val > ust or val < alt:
        plt.plot(0, val, "ko")
        x_offset = 0.2 if i % 2 == 0 else 0.25
        plt.text(x_offset, val, f"{val:.1f}", fontsize=8, color="black")

plt.ylabel("Horsepower")
plt.xticks([])
plt.show()


In [ ]:
# 4) Aykırı değerleri kaldırmak için aşağıdaki satırları yorumdan çıkarabilirsiniz
# veriSeti.drop(index=ust_aykiriDegerInd, inplace=True)
# veriSeti.drop(index=alt_aykiriDegerInd, inplace=True)



# **2.9 Örnekleme (Sampling ve Stratified Sampling)**

##2.9.1. Rastgele Örnekleme (Random Sampling)
bir veri kümesinden rastgele olarak veri noktaları seçmeyi ifade eder. Her bir veri noktasının seçilme şansı eşittir.
- **1'den 20'**ye kadar olan sayılar arasında rastgele **5** sayı seçmek istiyorsanız, bu sayılar her seferinde farklı olabilir.
- Örneğin: **[15, 6, 9, 14, 5]** ve başka bir seçimde farklı sayılar olabilir: **[16, 1, 17, 14, 10]**.

In [ ]:
# 1'den 20'ye kadar sayılar
listem = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
# diğer yol:  # listem = list(np.arange(1, 21))

# Rastgele 5 sayı seçimi
# sample() fonksiyonu ile her çağrıda farklı rastgele elemanlar seçilir ve tekrar eden sayı bulunmaz.
print("Örnek Seçim 1:", sample(listem, 5))
print("Örnek Seçim 2:", sample(listem, 5))


## 2.9.2. Sabit Tohumlu Rastgele Örnekleme (Seeded Sampling)
Rastgele örnekleme, her çalıştırıldığında farklı sonuçlar üretir. Ancak bazen belirli bir sonucu tekrarlamak isteriz. Bunun için bir "tohum" (seed) değeri kullanılır. Aynı tohum değeriyle rastgele seçim yapılırsa, her seferinde aynı seçimler elde edilir.

- raporlar yazarken ya da tekrarlanabilirlik isteyen deneyler yaparken faydalıdır.

In [ ]:
# Aynı seed ile benzer sonuçlar
random.seed(123)                                          # Algoritma için başlangıç noktasını (başlangıç değerini) 123 olarak belirler.
                                                          # Bu noktadan itibaren rastgele sayılar üretir.
print("Örnek Seçim 3 (seed=123):", sample(listem, 5))
random.seed(123)
print("Örnek Seçim 4 (seed=123):", sample(listem, 5))

random.seed(5)                                            # 5 yazarsanız başlangıç noktası (tohum değeri) değişir, dolayısıyla farklı sayılar üretirsiniz.
print("Örnek Seçim 5 (seed=5):", sample(listem, 5))
random.seed(5)
print("Örnek Seçim 6 (seed=5):", sample(listem, 5))


## 2.9.3. Yerine Koyarak Seçim (Sampling with Replacement)
Veri kümesindeki bir öğe birden fazla kez seçilebilir ve her seçimde aynı öğe tekrar kullanılabilir.
- Bir kutuda 10 top olduğunu düşünün. Bu kutudan 3 top seçiyorsunuz, ancak her seferinde topu kutuya geri koyuyorsunuz. Bu durumda aynı topu tekrar seçebilirsiniz.

In [ ]:
# Yerine koyarak seçim
print("Yerine koyarak seçim:", random.choices(listem, k=10))


##2.9.4. Eğitim ve Test Veri Seti Oluşturma
Eğitim ve test veri seti oluşturma, model eğitimi ve doğrulama işlemleri için veriyi ayırma işlemidir.
 - Genellikle verinin **%70'i eğitim seti**, **%30'u ise test seti** olarak ayrılır.
 - Eğitim seti, makine öğrenmesi modelinin eğitilmesi için kullanılırken, test seti modelin doğruluğunu ve başarısını değerlendirmek için kullanılır.

In [ ]:
egitim = veriSeti.sample(frac=0.7, replace=False, random_state=1)
      # sample(): Veri setinden rastgele örnekleme yapar.
      # frac=0.7 → veri setinin %70'ini seç,
      # replace=False → aynı satır birden fazla kez seçilmesin (tekrarsız seçim),
      # random_state=1 → her çalıştırmada aynı satırların seçilmesini sağlar (tekrarlanabilirlik için sabit tohum değeri)

ind = veriSeti.index.isin(egitim.index)
      # isin(): Eğitim setindeki index’lerin ana veri setinde olup olmadığını kontrol eder.
      # Sonuç olarak True/False değerlerinden oluşan bir dizi döner.

test = veriSeti[~ind]
      # ~ind: Eğitim setine dahil olmayan (False olan) satırları seçer.
      # Böylece kalan %30’luk kısım test setini oluşturur.

print("Eğitim Seti Boyutu:", egitim.shape)
    # shape: Seçilen veri çerçevesinin (DataFrame) satır ve sütun sayısını verir.

print("Test Seti Boyutu:", test.shape)

##2.9.5. Tabakalı Örnekleme (Stratified Sampling)
Tabakalı örnekleme, verinin belirli gruplara (tabakalara) ayrılarak her gruptan orantılı bir şekilde örnekleme yapılmasıdır. Özellikle dengesiz veri setlerinde, her sınıfın doğru bir şekilde temsil edilmesini sağlar.

- Tüm veri setindeki sınıf oranı → eğitim ve test setine eşit şekilde dağılır.
- Model eğitiminde azınlık sınıfların ihmal edilmesi önlenir.
- Eğitim ve test setlerinde "durum" değişkeninin dağılımı korunur.

Örneğin:
- Tüm veride **"Düşük: 208, Orta: 98, Yüksek: 92"** varsa,
- Eğitim setinde → **Düşük: 145, Orta: 69, Yüksek: 64**
- Test setinde → **Düşük: 63, Orta: 29, Yüksek: 28** şeklinde benzer oranlar korunur.

In [ ]:
egitim1, test1 = train_test_split(                         # train_test_split(): veri setini eğitim ve test setine ayırır
    veriSeti,                                              # veriSeti: bölünecek veri
    train_size=0.7,                                        # train_size=0.7 → verinin %70’i eğitim, %30’u test olacak şekilde ayır
    stratify=veriSeti["durum"],                            # stratify: "durum" değişkenine göre orantılı örnekleme yapılır
    random_state=1                                         # random_state=1 → aynı sonucu almak için sabit tohum (seed)
)

print("\n Durumu Dağılımı(durum %100):\n", veriSeti.durum.value_counts())    # value_counts(): "durum" değişkeninin sınıf dağılımını gösterir
print("\n Durumu Dağılımı(egitim1 %70):\n", egitim1.durum.value_counts())  # Eğitim setindeki sınıf dağılımını yazdırır
print("\n Durumu Dağılımı(test1 %20):\n", test1.durum.value_counts())      # Test setindeki sınıf dağılımını yazdırır
print(f"\n{'-'*82}\n")



















# ------------------------------------------------------------
# 🟩 5.GÖREV: guc_durumu değişkenine göre orantılı eğitim %80 - test % 20 ayırımı yapınız
# ------------------------------------------------------------
# 1. horsepower’dan üretilmiş guc_durumu sütununu kullanarak eğitim ve test setlerini oluşturun
# 2. train_size=0.8 → verinin %80’i eğitim, %20’si test olarak ayrılacaktır

# ------------------------------------------------------------



















# ------------------------------------------------------------
# 🟨 5.CEVAP ANAHTARI: guc_durumu değişkenine göre orantılı eğitim %80 - test % 20 ayırımı yapınız
# ------------------------------------------------------------


egitim2, test2 = train_test_split(                        # train_test_split(): veri setini eğitim ve test setine ayırır
    veriSeti,                                             # veriSeti: bölünecek veri
    train_size=0.8,                                       # train_size=0.8 → verinin %80’i eğitim, %20’u test olacak şekilde ayır
    stratify=veriSeti["guc_durumu"],                      # stratify: "guc_durumu" değişkenine göre orantılı örnekleme yapılır
    random_state=1                                        # random_state=1 → aynı sonucu almak için sabit tohum (seed)
)

print("\n Güç Durumu Dağılımı (guc_durumu %100):\n", veriSeti.guc_durumu.value_counts())
print("\n Güç Durumu Dağılımı(egitim2 %80):\n", egitim2.guc_durumu.value_counts())
print("\nGüç Durumu Dağılımı (test2 %20):\n", test2.guc_durumu.value_counts())






# **2.10 Yapay Kodlama (Label and Dummy Coding) ve Veri Normalizasyonu**

 ### 2.10.1. Kategorik Değişkeni Sayısal Forma Dönüştürme (cat.codes ve get_dummies)
 #### Neden Önemli?
Makine öğrenmesi ve istatistiksel modeller **sayısal verilerle çalışır**. Bu yüzden kategorik değişkenleri sayısal forma çevirmemiz gerekir.  
Bu işlem genellikle iki yöntemle yapılır:

---

#### 1️⃣ `cat.codes` → Label Encoding (Etiket Kodlama)

- Her kategoriye bir **sayısal değer** atanır.
- Tek bir sütun oluşturur.
Örnek:

| durum   | durum_s1 |
|---------|----------|
| Düşük   | 0        |
| Orta    | 1        |
| Yüksek  | 2        |

- ✅ Basit ve tek sütunludur.

---

#### 2️⃣ `get_dummies()` → Dummy (One-Hot) Encoding

- Her kategori için **ayrı bir sütun** oluşturur.
- 0 ve 1 değerleri içerir.
Örnek:

| Düşük | Orta | Yüksek |
|-------|------|--------|
| 1     | 0    | 0      |
| 0     | 1    | 0      |
| 0     | 0    | 1      |

- ✅ Her kategoriyi eşit ve bağımsız olarak gösterir.
- Özellikle **regresyon, mesafe temelli modeller** için daha uygundur.

---

#### Projede Neden Her İkisi de Kullanıldı?

Kodda hem `cat.codes` hem de `get_dummies()` kullanıldı. Bu sayede:

- `durum_s1` sütunu: **etiket kodlama** için,
- `Düşük`, `Orta`, `Yüksek` sütunları: **dummy kodlama** için oluşturulmuş oldu.

**Hangisi kullanılacak?**  
➡️ Modeliniz sıralı bilgiye ihtiyaç duymuyorsa dummy encoding daha güvenlidir.

---

✅ **Sonuç:**  
Kategorik veriyi sayısala çevirmek zorunludur. Ancak **nasıl çevireceğiniz**, kullandığınız algoritmaya bağlıdır.


In [ ]:
# Kategorik değişkeni sayısal forma dönüştürme
# 1️⃣ (cat.codes)
veriSeti["durum_s1"] = veriSeti["durum"].cat.codes                        # Kategorik 'durum' değişkenini 0,1,2 gibi sayılara çevir
print("durum_s1 değer dağılımı:", veriSeti["durum_s1"].value_counts())    # Sayısal karşılıkların frekansını yazdır

# 2️⃣ (get_dummies()
if not {'Düşük', 'Orta', 'Yüksek'}.issubset(veriSeti.columns):            # get_dummies() fonksiyonu her seferinde yeni sütunlar ekler. issubset ile tekrar eklenmesini önlemek gerekir.
    durum_s2 = pd.get_dummies(veriSeti["durum"], dtype=int)               # 'Düşük', 'Orta', 'Yüksek' gibi her sınıf için ayrı sütun
    veriSeti = pd.concat([veriSeti, durum_s2], axis=1)                    # Yeni dummy sütunlarını veriSeti'ne ekle
    print("Yapay Kodlama ile eklenen sütunlar:", list(durum_s2.columns))  # Eklenen sütun isimlerini yazdır

display(veriSeti.head(5))

















# ------------------------------------------------------------
# 🟩 6. GÖREV: Kategorik 'guc_durumu' değişkenini sayısal forma dönüştürünüz
# ------------------------------------------------------------
# Bu görevde, horsepower'dan elde edilen 'guc_durumu' değişkenini
# iki farklı yöntemle sayısallaştırmanız beklenmektedir:
#
# 1️⃣ Etiket kodlama (Label Encoding) yöntemiyle:
#     - .cat.codes kullanarak guc_durumu değişkenini 0, 1, 2 gibi sayılara çevirin.
#     - Yeni sütun adı: guc_durumu_s1
#     - Oluşan sayısal değerlerin frekans dağılımını yazdırınız.
#
# 2️⃣ One-hot kodlama (pd.get_dummies) yöntemiyle:
#     - 'Düşük Güç', 'Orta Güç', 'Yüksek Güç' sınıfları için ayrı sütunlar oluşturun.
#     - Dummy sütunları veriSeti'ne ekleyin.
#     - Eklenen sütun isimlerini ekrana yazdırınız.















# ------------------------------------------------------------
# 🟨 6.CEVAP ANAHTARI: Kategorik 'guc_durumu' değişkenini sayısal forma dönüştürme
# ------------------------------------------------------------

# 1️⃣ Etiket kodlama (cat.codes)
veriSeti["guc_durumu_s1"] = veriSeti["guc_durumu"].cat.codes                        # 'guc_durumu' değişkenini 0, 1, 2 gibi sayılara çevir
print("guc_durumu_s1 değer dağılımı:\n", veriSeti["guc_durumu_s1"].value_counts())  # Sayısal karşılıkların frekansını yazdır

# 2️⃣ One-hot kodlama (get_dummies)
if not {'Düşük Güç', 'Orta Güç', 'Yüksek Güç'}.issubset(veriSeti.columns):          # Dummy sütunlar daha önce eklenmediyse
    guc_durumu_s2 = pd.get_dummies(veriSeti["guc_durumu"], dtype=int)               # Her kategori için ayrı sütun oluştur
    veriSeti = pd.concat([veriSeti, guc_durumu_s2], axis=1)                         # Dummy sütunları veri setine ekle
    print("Yapay Kodlama ile eklenen sütunlar:", list(guc_durumu_s2.columns))       # Eklenen sütun isimlerini yazdır

display(veriSeti.head(5))





### 2.10.2. Min-Max Normalizasyonu (0-1 Arasına Sıkıştırma)
#### Neden Önemli?
Veri setindeki sayısal sütunlar farklı aralıklarda olabilir (örneğin: **mpg 10–40** arasında, **horsepower 50–200** arasında olabilir).
Bu fark, özellikle uzaklık/matris temelli modellerde (KNN, K-Means, Lojistik Regresyon vb.) bazı değişkenlerin modele daha fazla etki etmesine yol açar.

Bu nedenle, tüm değişkenlerin aynı ölçeğe getirilmesi gerekir.
Min-Max Normalizasyonu, her bir sayısal değeri 0 ile 1 arasına sıkıştırarak bu işlemi yapar.

**Avantajı:**
- Değişkenlerin eşit etki gücüne sahip olmasını sağlar.

- Özellikle mesafe tabanlı algoritmalar için önerilir.

In [ ]:
# Min-Max Normalizasyonu (verileri 0-1 arasına sıkıştırma)
veri = veriSeti.iloc[:, 0:8]  # İlk 8 sütunun sayısal olduğunu varsayarak seçiyoruz

scaler = MinMaxScaler()  # Min-Max ölçekleyici nesnesi oluştur
scaler.fit(veri)  # Minimum ve maksimum değerleri öğren
n_veriSeti = scaler.transform(veri)  # Veriyi 0-1 aralığına dönüştür
n_veriSeti = pd.DataFrame(n_veriSeti, columns=veri.columns)  # Sonuçları tekrar DataFrame'e çevir
print("\nMin-Max Normalizasyon sonrası ilk 5 satır:\n", n_veriSeti.head())
